## This script is meant to be run after a large set of simulations and will compile a spreadsheet listing cell shape and interactions for each simulation. (The only analzyed cells will be the central two in a row of four. Simulations with different initial setups will need to be analyzed differently.)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib notebook
import os
import time, sys
from IPython.display import clear_output
import math
import pickle
import pandas as pd
import matplotlib

plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
number_of_sides = 40
natural_cell_area = 1
natural_cell_radius = np.sqrt(natural_cell_area/np.pi)
natural_spacing = 2*np.pi*natural_cell_radius/number_of_sides

0.0886226925452758


In [3]:
def make_cells(**kwargs):
    np.random.seed(kwargs.get('seed', 2*21*1995))
    scatter = kwargs.get('scatter', natural_cell_radius*0.1)
    compression = kwargs.get('compression', False)
    theta = np.linspace(0, 2*np.pi, number_of_sides+1)[:-1].copy()
    list_of_cells = []
    for i in range(len(centers)):
        cell = np.zeros([number_of_sides, 2])
        cell[:,0] = natural_cell_radius*np.cos(theta)+centers[i,0]
        cell[:,1] = natural_cell_radius*np.sin(theta)+centers[i,1]
        cell = cell+scatter*np.random.random([number_of_sides,2])
        list_of_cells.append(cell)
    if not compression:
        left_min = np.min([-3, 2*(np.min(centers[:,0])-natural_cell_radius)])
        right_max = np.max([3, 2*(np.max(centers[:,0])+natural_cell_radius)])
    else:
        left_min = compression[0]
        right_max = compression[1]
    floorx = np.linspace(left_min, right_max, int((right_max-left_min)/natural_spacing))
    floorz = np.zeros_like(floorx)
    floor = np.array([floorx,floorz]).transpose()
    return floor, list_of_cells

def plot_cells(list_of_cells, **kwargs):
    connections = kwargs.get('connections',False)
    CCid = kwargs.get('CCid', 0.16)
    CSid = kwargs.get('CSid', 0.1)
    cell_cell, cell_cell_pairs = cell_cell_interactions(list_of_cells, CCid)
    sub_cell, sub_cell_pairs = cell_substrate_interactions(list_of_cells, CSid)
    left_min = -4.5
    right_max = 4.5
    top = 1.25*(np.max(centers[:,1])+natural_cell_radius)
    bottom = -0.2
    save_name = kwargs.get('save_name', False)
    fig = plt.figure(figsize = (12,12))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim([left_min,right_max])
    ax.set_ylim([bottom,top])
    for i in range(len(list_of_cells)):
        if connections == True:
            cell_i = list_of_cells[i]
            if len(sub_cell)>0:
                c_pts = cell_i[sub_cell_pairs[:,1].astype('int')[sub_cell==i]]
                s_pts = floor[sub_cell_pairs[:,0].astype('int')[sub_cell==i]]
                for j in range(len(c_pts)):
                    x = [c_pts[j][0], s_pts[j][0]]
                    y = [c_pts[j][1], s_pts[j][1]]
                    ax.plot(x,y, c = 'deepskyblue', linewidth = 2)
            for j in range(len(list_of_cells)):
                cell_j = list_of_cells[j]
                if len(cell_cell)>0:
                    con_ij = np.logical_and(cell_cell[:,0]==i, cell_cell[:,1]==j)
                    if np.sum(con_ij) > 0:
                        ci_pts = cell_i[cell_cell_pairs[:,0].astype('int')[con_ij]]
                        cj_pts = cell_j[cell_cell_pairs[:,1].astype('int')[con_ij]]
                        for k in range(len(ci_pts)):
                            x = [ci_pts[k][0],cj_pts[k][0]]
                            y = [ci_pts[k][1],cj_pts[k][1]]
                            ax.plot(x,y,c='darkgreen', linewidth = 3.5)
        ax.scatter(list_of_cells[i][:,0], list_of_cells[i][:,1], color = 'k')
        ax.plot(list_of_cells[i][:,0], list_of_cells[i][:,1], color = 'k')
        ax.plot([list_of_cells[i][0,0], list_of_cells[i][-1,0]], 
                [list_of_cells[i][0,1], list_of_cells[i][-1,1]], color = 'k')
    ax.scatter(floor, np.zeros_like(floor), color = 'r', alpha = 0.5)
    ax.plot(floor, np.zeros_like(floor), color = 'r', alpha = 0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    if save_name != False:
        plt.savefig(save_name+'.pdf', bbox_inches = 'tight', pad_inches = 1)
    return

def distance_matrix(xz1, xz2):
    mutx1 = np.outer(np.ones(len(xz2[:,0])), xz1[:,0])
    mutz1 = np.outer(np.ones(len(xz2[:,0])), xz1[:,1])
    mutx2 = np.outer(xz2[:,0], np.ones(len(xz1[:,0])))
    mutz2 = np.outer(xz2[:,1], np.ones(len(xz1[:,0])))
    return np.sqrt((mutx1-mutx2)**2+(mutz1-mutz2)**2).transpose()

def cell_substrate_interactions(list_of_cells, substrate_interaction_distance):
    all_distances = []
    cell_matrix_distances = []
    closest_cell_points = np.zeros([len(floor), len(list_of_cells)])
    closest_substrate_points = []
    i = 0
    for cell in list_of_cells:
        distances = distance_matrix(floor, cell)
        all_distances.append(distances)
        cell_matrix_distances.append(distances)
        closest_substrate_points.append(np.min(distances, axis = 0)<=substrate_interaction_distance)
        closest_cell_points[:,i] = np.min(distances, axis = 1)
        i+=1
    floor_interactions = np.min(closest_cell_points, axis = 1)<=substrate_interaction_distance
    floor_interaction_distances = closest_cell_points[floor_interactions]
   
    which_cell = np.zeros(np.sum(floor_interactions))
    which_pair = np.zeros([np.sum(floor_interactions),2])
    for j in range(len(floor_interactions[floor_interactions])):
        which_cell[j] = np.argwhere(floor_interaction_distances[j,:]==np.min(floor_interaction_distances[j,:]))[0,0]
        which_pair[j] = np.argwhere(cell_matrix_distances[int(which_cell[j])] == np.min(closest_cell_points, axis = 1)[floor_interactions][j])[0]
   
    which_cell_2 = []
    which_pair_2 = []
    for k in range(len(list_of_cells)):
        this_cell = which_cell==k
        this_cell_pairs = which_pair[this_cell]
        repeats = []
        for l in range(np.sum(this_cell)):
            if np.sum(this_cell_pairs[:,1]==this_cell_pairs[l,1])>1:
                repeats.append(this_cell_pairs[l,1])
            else:
                which_cell_2.append(k)
                which_pair_2.append(this_cell_pairs[l])
        repeats = np.sort(list(set(repeats)))
        for item in repeats:
            checks = this_cell_pairs[this_cell_pairs[:,1]==item]
            dists = []
            for m in range(len(checks)):
                dists.append(all_distances[k][int(checks[m][0]),int(checks[m][1])])
            keep = np.argwhere(dists == np.min(dists))[0,0]
            which_cell_2.append(k)
            which_pair_2.append(checks[keep])
               
   
    return np.array(which_cell_2), np.array(which_pair_2)

def cell_cell_interactions(list_of_cells, cell_cell_interaction_distance):
    center_positions = np.zeros([len(list_of_cells),2])
    for i in range(len(list_of_cells)):
        cell_centers = list_of_cells[i].mean(0)
        center_positions[i,0] = cell_centers[0]
        center_positions[i,1] = cell_centers[1]
    dist_bt_cell_centers = distance_matrix(center_positions,center_positions)
    pairs_of_cells_that_matter = np.argwhere(dist_bt_cell_centers>0)
    cells_that_matter = []
    for i in range(len(pairs_of_cells_that_matter)):
        if pairs_of_cells_that_matter[i,0]<pairs_of_cells_that_matter[i,1]:
            cells_that_matter.append(pairs_of_cells_that_matter[i])
    list_of_pairs = []
    for i in range(len(cells_that_matter)):
        cell_1 = list_of_cells[int(cells_that_matter[i][0])]
        cell_2 = list_of_cells[int(cells_that_matter[i][1])]
        distances = distance_matrix(cell_1,cell_2)
        closest_cell_cell_points = np.min(distances, axis = 1)
        cell_interactions = closest_cell_cell_points<=cell_cell_interaction_distance
        cell_interaction_distances = closest_cell_cell_points[cell_interactions]
        if np.sum(cell_interactions) == 0:
            which_pair = []
        else:
            which_pair = np.zeros([np.sum(cell_interactions),2])
            for j in range(np.sum(cell_interactions)):
                which_pair[j] = np.argwhere(distances == cell_interaction_distances[j])[0]
        list_of_pairs.append(which_pair)
    what_cells = []
    what_pairs = []
    which_cells = []
    which_pairs = []
    for i in range(len(cells_that_matter)):
        for j in range(len(list_of_pairs[i])):
            pairs = list_of_pairs[i][j]
            what_pairs.append(pairs)
            cells = [cells_that_matter[i]]
            what_cells.append(cells)
            which_pairs = np.vstack(what_pairs)
            which_cells= np.vstack(what_cells)

    if len(which_cells)>0:
        which_cells_2 = []
        which_pairs_2 = []
        for k in range(len(list_of_cells)):
            for n in range(len(list_of_cells)):
                these_cells = np.logical_and(which_cells[:,0]==k, which_cells[:,1]==n)
                these_cell_pairs = which_pairs[these_cells]
                repeats = []
                for l in range(np.sum(these_cells)):
                    if np.sum(these_cell_pairs[:,1]==these_cell_pairs[l,1])>1:
                        repeats.append(these_cell_pairs[l,1])
                    else:
                        which_cells_2.append([k,n])
                        which_pairs_2.append(these_cell_pairs[l])
                repeats = np.sort(list(set(repeats)))
                for item in repeats:
                    checks = these_cell_pairs[these_cell_pairs[:,1]==item]
                    dists = []
                    for m in range(len(checks)):
                        dists.append(distance_matrix(list_of_cells[k], list_of_cells[n])[int(checks[m][0]),int(checks[m][1])])
                    keep = np.argwhere(dists == np.min(dists))[0,0]
                    which_cells_2.append([k,n])
                    which_pairs_2.append(checks[keep])
    else:
        which_cells_2 = which_cells
        which_pairs_2 = which_pairs
       
    if len(np.array(which_cells_2))>0:
        return np.concatenate([np.array(which_cells_2), np.array(which_cells_2)[:,[1,0]]]), np.concatenate([np.array(which_pairs_2), np.array(which_pairs_2)[:,[1,0]]])
    else:
        return np.array(which_cells_2), np.array(which_pairs_2)

def area(cell):
    x = cell[:,0]
    z = cell[:,1]
    return 0.5*np.sum(x*np.roll(z,-1)-z*np.roll(x,-1))

def perimeter(cell):
    ls = np.sum(np.sqrt(np.sum((np.roll(cell, -1, axis = 0) - cell)**2, axis = 1)))
    return ls

def circularity(cell):
    circ = (4*np.pi*area(cell))/((perimeter(cell))**2)
    return circ

def average_area_center_cells(list_of_cells):
    list_of_areas = []
    for cell in list_of_cells[1:-1]:
        list_of_areas.append(area(cell))
    return (np.mean(list_of_areas), np.std(list_of_areas))

def average_perimeter_center_cells(list_of_cells):
    list_of_perimeters = []
    for cell in list_of_cells[1:-1]:
        list_of_perimeters.append(perimeter(cell))
    return (np.mean(list_of_perimeters), np.std(list_of_perimeters))

def average_circularity_center_cells(list_of_cells):
    list_of_circ = []
    for cell in list_of_cells[1:-1]:
        list_of_circ.append(circularity(cell))
    return (np.mean(list_of_circ), np.std(list_of_circ))

def percent_cc_adhesions_center_cells(list_of_cells, cell_interaction_distance):
    list_of_percents = []
    for i in range(len(list_of_cells[1:-1])):
        cell = list_of_cells[i+1]
        list_of_percents.append(np.sum(cell_cell_interactions(list_of_cells,cell_interaction_distance)[0]==(i+1))/(2*len(cell)))
    return (np.mean(list_of_percents), np.std(list_of_percents))

def percent_cs_adhesions_center_cells(list_of_cells, substrate_interaction_distance):
    list_of_percents = []
    for i in range(len(list_of_cells[1:-1])):
        cell = list_of_cells[i+1]
        list_of_percents.append(np.sum(cell_substrate_interactions(list_of_cells,substrate_interaction_distance)[0]==(i+1))/(len(cell)))
    return (np.mean(list_of_percents), np.std(list_of_percents))

In [9]:
num_DENSITY = 5
num_i = 4
num_j = 5
num_k = 7

In [10]:
gCS = [0.001,0.002,0.005,0.01,0.02,0.05,0.1]
compressions = [[-10,10],[-3.6,3.6],[-3.2,3.2],[-2.8,2.8],[-2.4,2.4]]
centers_un = [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_4 =  [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_3 =  [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_2 =  [[[-2.4,1.6], [-0.8,1.5], [0.8,1.5], [2.4,1.5]],
              [[-2.4,1.5], [-0.82,1.4], [0.8,1.5], [2.4,1.5]],
              [[-2.4,1.5], [-0.8,1.5], [0.81,1.7], [2.4,1.5]],
              [[-2.4,1.5], [-0.8,1.5], [0.8,1.5], [2.35,1.4]]]
centers_1 =  [[[-2.1,1.6], [-0.7,1.5], [0.7,1.5], [2.1,1.5]],
              [[-2.1,1.5], [-0.72,1.4], [0.7,1.5], [2.1,1.5]],
              [[-2.1,1.5], [-0.7,1.5], [0.71,1.7], [2.1,1.5]],
              [[-2.1,1.5], [-0.7,1.5], [0.7,1.5], [2.05,1.4]]]
configurations = [centers_un,centers_4,centers_3,centers_2,centers_1]
seeds = [2*21*95, 11*4*96, 5*9*64, 13, 42]

In [12]:
directory = 'Cell-Substrate/'
extracted_data = np.zeros([num_i*num_j*num_k*num_DENSITY,9])
spot = 0
for DENSITY in range(num_DENSITY):
    for i in range(num_i):
        centers = np.array(configurations[DENSITY][i])
        for j in range(num_j):
            floor, list_of_cells = make_cells(seed = seeds[j], compression = compressions[DENSITY])
            for k in range(num_k):
                name = 'Run_density='+str(DENSITY)+'_i='+str(i)+'_j='+str(j)+'_k='+str(k)+'.pkl'
                with open(directory+name, 'rb') as g:
                    run = pickle.load(g)[-1]
                a, da = average_area_center_cells(run)
                p, dp = average_perimeter_center_cells(run)
                c, dc = average_circularity_center_cells(run)
                cc, dcc = percent_cc_adhesions_center_cells(run, 0.16)
                cs, dcs = percent_cs_adhesions_center_cells(run, 0.1)
                extracted_data[spot,:] = [DENSITY, i, j, k, a, p, c, cc, cs]
                print(spot)
                spot+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
df = pd.DataFrame(extracted_data, columns = ['DENSITY','i','j','k','area','perimeter','circularity','percentage_cell_contacts','percentage_substrate_contacts'])
df.to_csv('Cell-Substrate Analyzed.csv')

In [61]:
num_DENSITY = 5
num_i = 4
num_j = 5
num_k = 5

In [62]:
gCC = [0.00001, 0.0001, 0.001, 0.01, 0.1]
compressions = [[-5,5],[-3.6,3.6],[-3.2,3.2],[-2.8,2.8],[-2.4,2.4]]
centers_un = [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_4 =  [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_3 =  [[[-2.7,1.6], [-0.9,1.5], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.92,1.4], [0.9,1.5], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.91,1.7], [2.7,1.5]],
              [[-2.7,1.5], [-0.9,1.5], [0.9,1.5], [2.65,1.4]]]
centers_2 =  [[[-2.4,1.6], [-0.8,1.5], [0.8,1.5], [2.4,1.5]],
              [[-2.4,1.5], [-0.82,1.4], [0.8,1.5], [2.4,1.5]],
              [[-2.4,1.5], [-0.8,1.5], [0.81,1.7], [2.4,1.5]],
              [[-2.4,1.5], [-0.8,1.5], [0.8,1.5], [2.35,1.4]]]
centers_1 =  [[[-2.1,1.6], [-0.7,1.5], [0.7,1.5], [2.1,1.5]],
              [[-2.1,1.5], [-0.72,1.4], [0.7,1.5], [2.1,1.5]],
              [[-2.1,1.5], [-0.7,1.5], [0.71,1.7], [2.1,1.5]],
              [[-2.1,1.5], [-0.7,1.5], [0.7,1.5], [2.05,1.4]]]
configurations = [centers_un,centers_4,centers_3,centers_2,centers_1]
seeds = [2*21*95, 11*4*96, 5*9*64, 13, 42]

In [64]:
directory = 'Cell-Cell/'
extracted_data = np.zeros([num_i*num_j*num_k*num_DENSITY,9])
spot = 0
for DENSITY in range(num_DENSITY):
    for i in range(num_i):
        centers = np.array(configurations[DENSITY][i])
        for j in range(num_j):
            floor, list_of_cells = make_cells(seed = seeds[j], compression = compressions[DENSITY])
            for k in range(num_k):
                name = 'Run_density='+str(DENSITY)+'_i='+str(i)+'_j='+str(j)+'_k='+str(k)+'.pkl'
                with open(directory+name, 'rb') as g:
                    run = pickle.load(g)[-1]
                a, da = average_area_center_cells(run)
                p, dp = average_perimeter_center_cells(run)
                c, dc = average_circularity_center_cells(run)
                cc, dcc = percent_cc_adhesions_center_cells(run, 0.16)
                cs, dcs = percent_cs_adhesions_center_cells(run, 0.1)
                extracted_data[spot,:] = [DENSITY, i, j, k, a, p, c, cc, cs]
                print(spot)
                spot+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [65]:
df = pd.DataFrame(extracted_data, columns = ['DENSITY','i','j','k','area','perimeter','circularity','percentage_cell_contacts','percentage_substrate_contacts'])
df.to_csv('Cell-Cell Analyzed.csv')